# How to use *FastCORE* with *Troppo*

A typical workflow follows two main steps. The first is to attribute a score to each reaction of the model, in accordance with the omics data imputed. The second is to use the scores and apply an integration method to select a subset of reactions to build the final model.

Integration scoring methods implemented in *Troppo* are:
- continuous: `ContinuousScoreIntegrationStrategy`
- threshold: `ThresholdSelectionIntegrationStrategy`
- default_core: `DefaultCoreIntegrationStrategy`
- adjusted_score: `AdjustedScoreIntegrationStrategy`
- custom: `CustomSelectionIntegrationStrategy`

Omics integration methods implemented in *Troppo* are:
- gimme: `GIMME`
- tinit: `tINIT`
- fastcore: `FastCORE`
- imat: `IMAT`
- swiftcore: `SWIFTCORE`
- corda: `CORDA`

This example can be applied for all the Omics integration methods implemented in this package. Note that the appropriate integration scoring method can differ between integration algorithms. For instance, for *GIMME* a continuous scoring method can be used, while for `fastcore` a threshold scoring method is more adequate.

### Imports and Setup

In [1]:
import pandas as pd
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper
from troppo.omics.integration import CustomSelectionIntegrationStrategy
from troppo.methods.reconstruction.fastcore import FASTcore, FastcoreProperties

C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\wrappers\external_wrappers.py:12: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  reader can also be created using the get_model_reader function on cobamp.wrappers''')


Define the parsing rules for the GPRs that will be used later on.

In [2]:
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

### Read model and omics data

In [3]:
model = cobra.io.read_sbml_model('data/HumanGEM_Consistent_COVID19_HAM.xml')
model

Name,HumanGEM
Memory address,17170e8f488
Number of metabolites,6149
Number of reactions,10347
Number of genes,2976
Number of groups,142
Objective expression,1.0*biomass_human - 1.0*biomass_human_reverse_fb2f2
Compartments,"Cytosol, Lysosome, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [4]:
omics_data = pd.read_csv(filepath_or_buffer='data/Desai-GTEx_ensembl.csv', index_col=0)

### Create a container for the omics data.

The `TabularReader` class is used to read and store the omics data in a container that can then be used by *Troppo*. 

Relevant arguments from the `TabularReader` class:
- `path_or_df`: the omics data can be either a pandas dataframe or a path to a dataset file. The file can be in any format supported by pandas.
- `index_col`: the name of the column that contains the identifiers of the genes.
- `sample_in_rows`: a boolean indicating whether the samples are in rows or columns.
- `header_offset`: the number of rows to skip before reading the header.
- `omics_type`: a string containing the type of omics data. This is used to select the appropriate integration method.
- `nomenclature`: a string containing the nomenclature of the identifiers in the omics data. This is used to map the identifiers to the identifiers in the model.

The `to_containers()` method returns a list of containers, one for each sample of the dataset. In this example, we will be using only one sample, however, the process can be iterated for all the samples in the dataset.
The `get_integrated_data_map()` method is used to map the identifiers in the omics data to the identifiers in the model. This is done by using the `gpr_gene_parse_function` argument from the `ModelBasedWrapper` class.

In [5]:
omics_container = TabularReader(path_or_df=omics_data, nomenclature='ensemble_gene_id', omics_type='transcriptomics').to_containers()[0]
omics_container

### Create a model wrapper.

The `ModelBasedWrapper` class is used to wrap the model so that it can be used by *Troppo*.

Relevant arguments from this class include:
- `model`: the model to be wrapped.
- `ttg_ratio`: the ratio between the number of reactions to be selected and the total number of reactions in the model.
- `gpr_gene_parse_function`: a function that parses the GPRs of the model. This is used to map the identifiers in the omics data to the identifiers in the model.

Important attributes from this class include:
- `model_reader`: a COBRAModelObjectReader instance containing all the information of the model, such as, reaction_ids, metabolite_ids, GPRs, bounds, etc.
- `S`: the stoichiometric matrix of the model.
- `lb`: the lower bounds of the reactions in the model.
- `ub`: the upper bounds of the reactions in the model.

In [6]:
model_wrapper = ModelBasedWrapper(model=model, ttg_ratio=9999, gpr_gene_parse_function=replace_alt_transcripts)
model_wrapper

C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000131069
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111058 or _ENSG00000154930
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000091140 and _ENSG00000110435 and _ENSG00000131828 and _ENSG00000150768 and _ENSG00000163114 and _ENSG00000168291
  warnings.warn('Could not normalize this rule: ' + rule)
C:\Users\biosy\anaconda3\envs\s2m2\lib\site-packages\cobamp\gpr\core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111716 or _ENSG00000134333 or _ENSG00000151116 or _ENSG00000166796 or _ENSG00000166800 or _ENSG00000171989
  warnings.warn('Could not normalize this rule: ' + r

### Map the identifiers in the omics data to the identifiers in the model

For this we can use the `get_integrated_data_map()` method from the `TabularReader` class. This maps the gene ids in the omics dataset reaction ids in the model through their GPRs, and attributes a score to each reaction in accordance with the expression values of the associated genes. This method returns a dictionary with the reaction ids as keys and the scores as values.

Important arguments from this method include:
- `model_reader`: a COBRAModelObjectReader instance containing all the information of the model. It can be accessed through the `model_wrapper.model_reader`.
- `and_func`: a function that is used to combine the scores of the genes associated with a reaction for AND rules in the GPR. In this example, we will be using the minimum function, which means that the score of a reaction with AND in their GPRs will be the minimum score of the genes associated with it.
- `or_func`: a function that is used to combine the scores of the genes associated with a reaction for OR rules in the GPR. In this example, we will be using the sum function, which means that the score of a reaction with OR in their GPRs will be the sum of the scores of the genes associated with it.

In [7]:
data_map = omics_container.get_integrated_data_map(model_reader=model_wrapper.model_reader, and_func=min, or_func=sum)

### Integrate scores

The `CustomSelectionIntegrationStrategy` class allows the user to define a custom function that is tailored to the output we need for the following steps of the pipeline.
For the `FastCORE` method the most adequate integration strategy is to select reactions whose score is above a defined threshold.

This also can be achieved by using the `ThresholdSelectionIntegrationStrategy` class, however, since we also want to include a set of reaction to be protected during the integration we will use a custom method that will be defined by `integration_fx`.

Moreover, through this function we also want the output to be a list with reaction IDs that will belong to the core reactions that will be inputted for the `FastCORE` algorithm.


In [8]:
from math import log
threshold =  (5 * log(2))
protected_reactions = ['biomass']

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > threshold) or k in protected_reactions]]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])
threshold_scores = threshold_integration.integrate(data_map=data_map)

print(threshold_scores)

[['HMR_4099', 'HMR_4281', 'HMR_4388', 'HMR_4283', 'HMR_8357', 'HMR_4379', 'HMR_4301', 'HMR_4355', 'HMR_4358', 'HMR_4363', 'HMR_4365', 'HMR_4368', 'HMR_4370', 'HMR_4371', 'HMR_4372', 'HMR_4373', 'HMR_4375', 'HMR_4377', 'HMR_4381', 'HMR_4394', 'HMR_4396', 'HMR_4521', 'HMR_6410', 'HMR_7745', 'HMR_7746', 'HMR_7747', 'HMR_7748', 'HMR_7749', 'HMR_5395', 'HMR_5396', 'HMR_9727', 'HMR_5397', 'HMR_5398', 'HMR_5401', 'HMR_4128', 'HMR_4130', 'HMR_4414', 'HMR_4774', 'HMR_4775', 'HMR_7674', 'HMR_8766', 'HMR_4297', 'HMR_4316', 'HMR_4319', 'HMR_4383', 'HMR_4385', 'HMR_4386', 'HMR_4387', 'HMR_4399', 'HMR_4490', 'HMR_4706', 'HMR_4590', 'HMR_4591', 'HMR_4592', 'HMR_8344', 'HMR_8352', 'HMR_8727', 'HMR_6537', 'HMR_1568', 'HMR_3853', 'HMR_3854', 'HMR_3855', 'HMR_3857', 'HMR_3859', 'HMR_4087', 'HMR_4091', 'HMR_4103', 'HMR_4193', 'HMR_8497', 'HMR_8500', 'HMR_8501', 'HMR_8502', 'HMR_8503', 'HMR_8504', 'HMR_8506', 'HMR_8507', 'HMR_8508', 'HMR_8509', 'HMR_8511', 'HMR_8514', 'HMR_4280', 'HMR_0153', 'HMR_3212', 'H

### Run the FastCORE algorithm

The `FastcoreProperties` class is used to create the properties for the GIMME algorithm. This class contains the following arguments:
- `core`: List of indexes of the reactions that are considered core, as determined by the integrated scores.
- `flux_threshold`: Flux threshold for the algorithm.
- `solver`: Solver to be used.

The `FASTcore` class is used to run the GIMME algorithm. This class contains the following arguments:
- `S`: the stoichiometric matrix of the model. It can be accessed through the `model_wrapper.S`.
- `lb`: the lower bounds of the reactions in the model. It can be accessed through the `model_wrapper.lb`.
- `ub`: the upper bounds of the reactions in the model. It can be accessed through the `model_wrapper.ub`.
- `properties`: a `FastcoreProperties` instance containing the properties for the GIMME algorithm.

In the end, the `run()` method of the `FASTcore` class will return a list with the index of the reactions to be kept in the model.

In [18]:
# Get the index of the reaction of the CORE reaction set
ordered_ids = {r:i for i,r in enumerate(model_wrapper.model_reader.r_ids)}
core_idx = [[ordered_ids[k] for k in l] for l in threshold_scores]

# Define the FastCORE properties
properties = FastcoreProperties(core=core_idx, solver='CPLEX')

# instantiate the FastCORE class
fastcore = FASTcore(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

# Run the algorithm
model_fastcore = fastcore.run()

print(model_fastcore)

J size2328
[    1     7     8 ... 10232 10313 10319]
before LP7
LP7
-0.23279999999999068
done LP7
LP9
830842041.1604956
Warning, Solution is not optimal
done LP9
128 5125
before LP7
LP7
-0.008600000000000003
done LP7
LP9
1469.99999999984
Warning, Solution is not optimal
done LP9
128 5364
before LP7
LP7
0.0
done LP7
28 5364
Flipped
before LP7
LP7
-0.0018000000000000006
done LP7
LP9
390.00000000186265
Warning, Solution is not optimal
done LP9
28 5421
before LP7
LP7
0.0
done LP7
10 5421
Flipped
before LP7
LP7
0.0
done LP7
10 5421
before LP7
LP7
-0.0001
done LP7
LP9
20.0
Warning, Solution is not optimal
done LP9
10 5425
before LP7
LP7
-0.0001
done LP7
LP9
20.0
Warning, Solution is not optimal
done LP9
8 5429
before LP7
LP7
-0.0001
done LP7
LP9
20.0000001385808
Warning, Solution is not optimal
done LP9
6 5433
before LP7
LP7
-0.0001
done LP7
LP9
20.0
Warning, Solution is not optimal
done LP9
4 5439
0 5439
[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 2